### 0) Data Importing and Helper Function Definition:

In [ ]:
import numpy as np

DATA_folder  = '../../Data/'
data = np.load(DATA_folder+'train_imgs.npy')
lbls = np.load(DATA_folder+'train_lbls.npy')
test_data = np.load(DATA_folder+'test_imgs.npy')
test_lbls = np.load(DATA_folder+'test_lbls.npy')

### 1) Definition of RBF Transformer Object:

RBF Transformer is a scikit-learn compatible transformer object that implements:

    - fit method       - clusters the digit-separated data, and computes cluster centers and inv. sq. deviations
    - transform method - based on obtained centers and deviations it computes the fi values (RBF layer outputs) as 
                         simple Gaussian functions -> Fi_k(x) = exp{ -sqrt( sum[ (x_i-c_k_i)/(dev_i^2) ] ) } 
                         
## Normalizing each Fi row with row sum -> STABILIZED the Regression a lot!!!

## For PCA fit_transform() method needs to be separated to fit() and then transform()!!!

#### Definition of Clustering Function:

In [ ]:
### Clustering:

# from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.cluster import KMeans, SpectralClustering
import random

# clust = AgglomerativeClustering(n_clusters=500, linkage='complete')
# cluster_labels = clust.fit_predict(dig_data)

# Function for data clustering, and computation of cluster center vectors and inv. sq. deviation vectors
def form_clusters(data, n_kmeans, n_agglo, n_random, dig):
    
    n_km = min(n_kmeans,data.shape[0])
    n_ag = min(n_agglo ,data.shape[0])
    n_rn = min(n_random,data.shape[0])
    
    lbls_set = []
    
    if n_km>0:
        kmeans = KMeans(n_clusters=n_km, random_state=0, init='k-means++', algorithm='elkan')
        lbls_kmeans = kmeans.fit_predict(data)
        lbls_set.append(lbls_kmeans)
    
    if n_ag>0:
        agglo  = SpectralClustering(n_clusters=n_ag, affinity='nearest_neighbors',eigen_solver='arpack',random_state=456,assign_labels='discretize')
#         lbls_agglo  = agglo.fit_predict(np.exp(-data / data.std()))
        lbls_agglo  = agglo.fit_predict(data)
        lbls_set.append(lbls_agglo)
    
    centers     = []
    
    # Find cluster centers and covar matrix:
    for lbls in lbls_set:
        for k in range(max(lbls)+1):
            cluster = data[lbls==k,:]
            centers.append(cluster.mean(axis=0))
            
        del cluster
        
    # Add random points as centers:
    if n_rn>0:
        # seed a new random generator (to get repeatable results for hyperparm tuning)
        prng = np.random.RandomState()
        prng.seed(dig*n_rn)
        smpls = prng.choice(data.shape[0],n_rn,replace=False)
        for s in smpls:
            centers.append(data[s,:])
        
    return centers;


#### Definition of RBF (Fi) Function:

In [ ]:
import gc

# chunks of 100, (10k smpls, 500 centers) ->  110.8s
# chunks of 200, (10k smpls, 500 centers) ->  102.3s
# chunks of 250, (10k smpls, 500 centers) ->  115.6s
# chunks of 500, (10k smpls, 500 centers) ->  145.1s

# Function to compute whole Fi output for given dataset (for all RB centers)
def fi_transform(data, all_centers):
    # data        - given dataset matrix for which to compute fi values
    # center      - list of all center vectors on which to compute fi vals

    new_data = np.empty((data.shape[0],len(all_centers)))
    st = 0
    ch = 200
    ns = data.shape[0]
    # Process data in chunks of 200 smpls (optimal speed)
    while st<ns:
        en = min(st+ch,ns)
        for k in range(len(all_centers)):
            # # SAME DEVIATION for ALL DIMENSIONS (better much better than cluster separate):
            new_data[st:en,k] = (np.square(data[st:en] - np.repeat(all_centers[k][np.newaxis,:],data[st:en].shape[0],axis=0))).mean(axis=1)
        st = en
        gc.collect()
    new_data = np.exp(-np.sqrt(new_data))
    return new_data

#### Definition of RBF Transformer (sklearn compatible object):

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import time

# SKLEARN Compatible Transformer - supports fit method (custering data) and transform method (calculating fi values)
class myRBFtransformer(BaseEstimator, TransformerMixin):
    
    # Transformer initialization (default to 50 kMeans clusters)
    def __init__(self, n_centers=250, rn_ratio=0.1, cl_ratio=0.5, debug=False):
        self.n_centers = n_centers              # number of data centers to be formed per digit
        self.rn_ratio  = min(1,max(0,rn_ratio)) # random  ratio (between Random Selection and Clustering)
        self.cl_ratio  = min(1,max(0,cl_ratio)) # cluster ratio (between kMeans and Agglomerative)
        self.debug     = debug                  # debug flag
        if self.debug:
            print(self.n_centers,self.cl_ratio)
    
    # Clusters each digit and finds cluster centers:
    def fit(self, X, y):
        
        self.centers     = [] # list of cluster center vectors
        self.num_centers = [] # list of number of centers per digit
        
        # Calc. num. of clusters per digit based on assigned ratio (ratio*(num of smpls / 10))
        n_random   = round(self.rn_ratio*self.n_centers)
        n_clusters = self.n_centers - n_random
        n_kmeans   = round(self.cl_ratio*n_clusters)
        n_agglo    = n_clusters - n_kmeans
        
        if self.debug:
            print('Clustering data ',(n_kmeans,n_agglo))
            t = time.time()
        
        # Cluster the data over each digit
        for dig in range(10):
            # print('Clustering digit: ',dig)
            data = X[y==dig,:]
            centers = form_clusters(data, n_kmeans, n_agglo, n_random, dig)
            self.centers.extend(centers)
            self.num_centers.append(len(centers))
            
        if self.debug:
            print('Clustering time: ',(time.time()-t))
        
        return self
    
    # Computes fi values (with Gaussian function) based on obtained data centers
    def transform(self, X, y=None):
        
        # Compute all Fi values:
        if self.debug:
            print('Calculating all Fi outputs !!!')
            t = time.time()
        all_fis = fi_transform(X, self.centers)
        
        if self.debug:
            print('Fi calculation time: ',time.time()-t)
        
        return all_fis

In [ ]:
# # Test:
# rbf = myRBFtransformer(n_kmeans = 10)
# a   = rbf.fit_transform(X=data[:500,:],y=lbls[:500])
# a.max(axis=1)

### 3) Pipelining the Model:

In [ ]:
import types
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model  import LogisticRegression

n_pca     = 545 # First dimension to retain 99 % variance (stanford edu recomendation for images)!!!
n_centers = 50
rn_ratio  = 0.3
cl_ratio  = 0.3
C         = 7500
debug     = False


scaler = StandardScaler()
pca    = PCA(n_components=n_pca, whiten=True, random_state=1234)
rbf    = myRBFtransformer(n_centers=n_centers,rn_ratio=rn_ratio,cl_ratio=cl_ratio, debug=debug)
logreg = LogisticRegression(tol=1e-12,C=C,random_state=12,solver='liblinear')

############################################################################
# For PCA -> fit_transform() method needs to be modified so it calls fit() #
#            and transform() methods separately (not at the same time) !!! #
#                                                                          #
#         -> otherwise it will produce slightly different outputs when     #
#            transform() is called next time (comp. to fit_transform())    #
#                                                                          #
#            WHICH massively messes up with the OUTPUT of FI LAYER !!!!    #
############################################################################

def new_fit_transform(self, X, y=None, **fit_params):
    if y is None:
        # return self.fit(X, **fit_params).transform(X)
        self.fit(X, **fit_params)
        return self.transform(X)
    else:
        # return self.fit(X, y, **fit_params).transform(X)
        self.fit(X, y, **fit_params)
        return self.transform(X)
    
pca.fit_transform = types.MethodType(new_fit_transform, pca)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

pipe = Pipeline(steps=[('scal', scaler), ('pca', pca), ('rbf',rbf), ('logreg',logreg)])

In [ ]:
# pipe1 = Pipeline(steps=[('scal', scaler), ('pca', pca)])
# new_data = pipe1.fit_transform(test_data)
# rbf.fit(new_data, test_lbls)
# import time
# t = time.time()
# new_data_2 = rbf.transform(new_data)
# print(time.time()-t)

In [ ]:
# my_data = test_data[:10000]
# my_lbls = test_lbls[:10000]
# pipe.fit(data,lbls)

# print(pipe.score(my_data,my_lbls)*100)

In [ ]:
# from sklearn.model_selection import GridSearchCV
from dask_searchcv import GridSearchCV

Ks  = [50]
RNs = [0.3] # Random Ratio
CLs = [0.3] # Cluster Ratio
Cs  = [7.5e3]

fld = 5
dt  = 10000
param_grid = [
    {
        'rbf__n_centers': Ks,
        'rbf__rn_ratio':  RNs,
        'rbf__cl_ratio':  CLs,
        'logreg__C':      Cs
    }
]

In [ ]:
from sklearn.model_selection import PredefinedSplit
import gc

test_fold = [-1]*data.shape[0]+[0]*test_data.shape[0]
ps = PredefinedSplit(test_fold)

tot_data = np.vstack((new_data,new_test_data))
tot_lbls = np.vstack((lbls,test_lbls))

del data,test_data,lbls,lbls_data,test_fold
gc.collect()

In [ ]:
grid = GridSearchCV(pipe, cv=ps, n_jobs=-1, param_grid=param_grid)

In [ ]:
from dask.diagnostics import ProgressBar

In [ ]:
# # Fit on train data and check with test data:
# n_centers = 50; rn_ratio = 0.3; cl_ratio = 1.0; C = 7500; - 91.91%
# # Fit on train data and check with test data (spectral):
# n_centers = 50; rn_ratio = 0.3; cl_ratio = 0.3; C = 7500; - 92.05%


In [ ]:
with ProgressBar():
    #grid.fit(data, lbls)
    grid.fit(tot_data, tot_lbls)
    
print(max(grid.cv_results_['mean_test_score'])*100)

In [ ]:
print(grid.best_params_)

In [ ]:
print(max(grid.cv_results_['mean_test_score'])*100)

In [ ]:
print(grid.cv_results_['mean_test_score'])

In [ ]:
import types
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model  import LogisticRegression

n_pca     = 545 # First dimension to retain 99 % variance (stanford edu recomendation for images)!!!
n_centers = 50
rn_ratio  = 0.3
cl_ratio  = 0.3
C         = 7500
debug     = True

scaler = StandardScaler()
pca    = PCA(n_components=n_pca, whiten=True, random_state=1234)
rbf    = myRBFtransformer(n_centers=n_centers,rn_ratio=rn_ratio,cl_ratio=cl_ratio, debug=debug)
logreg = LogisticRegression(tol=1e-12,C=C,random_state=12,solver='liblinear')

def new_fit_transform(self, X, y=None, **fit_params):
    if y is None:
        # return self.fit(X, **fit_params).transform(X)
        self.fit(X, **fit_params)
        return self.transform(X)
    else:
        # return self.fit(X, y, **fit_params).transform(X)
        self.fit(X, y, **fit_params)
        return self.transform(X)
    
pca.fit_transform = types.MethodType(new_fit_transform, pca)

In [ ]:
from sklearn.model_selection import PredefinedSplit

test_fold = [-1]*data.shape[0]+[0]*test_data.shape[0]
ps = PredefinedSplit(test_fold)

In [ ]:
from sklearn.pipeline import Pipeline

pipe1 = Pipeline(steps=[('scal', scaler), ('pca', pca), ('rbf',rbf)])
pipe1.fit(data,lbls)


In [ ]:
import gc

new_data      = pipe1.transform(data)
new_test_data = pipe1.transform(test_data)

tot_data = np.vstack((new_data,new_test_data))
tot_lbls = np.hstack((lbls,test_lbls))

del data,test_data,new_data,new_test_data,lbls,test_lbls
gc.collect()

In [ ]:
pipe2 = Pipeline(steps=[('logreg',logreg)])
Cs  = [1e5,1e6]

param_grid = [
    {
        'logreg__C': Cs
    }
]

In [ ]:
from dask_searchcv import GridSearchCV
grid = GridSearchCV(pipe2, cv=ps, n_jobs=-1, param_grid=param_grid)


In [ ]:
# Cs  = [1,1e1,1e2,1e3,7.5e3,1e4] - 1e4 - best at 92.06 %
# Cs  = [1e5,1e6] - [0.9218,0.9227] - 92.27% but much much slower



In [ ]:
from dask.diagnostics import ProgressBar
with ProgressBar():
    grid.fit(tot_data, tot_lbls)
    
print(max(grid.cv_results_['mean_test_score'])*100)

In [ ]:
print(grid.best_params_)
print(grid.cv_results_['mean_test_score'])